# binder sort order

script for generating the sort order of cards to be put into my binders

## imports and environment setting

In [ ]:
import sys, os

sys.path.insert(0, os.path.realpath('../'))

In [ ]:
import mtg.extract.tappedout as T

In [ ]:
url = T._INVENTORY_URL
owner = 'ndlambo'

In [ ]:
import importlib
importlib.reload(T)

In [ ]:
T._logging.basicConfig()

In [ ]:
bs = T.binder_summary(url, owner, mainthresh=0.9)

In [ ]:
import math

import ipywidgets

from pprint import pprint

In [ ]:
page_size = 9
num_pages = math.ceil(bs.shape[0] / page_size)
@ipywidgets.interact(page=ipywidgets.IntSlider(values=0, min=0, max=num_pages))
def foo(page):
    print(f'page {page}:')
    return (bs
            .iloc[page_size * page: page_size * (page + 1)]
           [['num_unclaimed', 'name', 'set', 'qty', 'colorIdentity',
             'foil', 'mytype', 'convertedManaCost', 'price',
             'in_collections']]
           .rename(columns={'convertedManaCost': 'cmc'}))

In [ ]:
page_size = 9
num_pages = math.ceil(bs.shape[0] / page_size)
@ipywidgets.interact(page=ipywidgets.IntSlider(values=0, min=0, max=num_pages))
def foo(page):
    print(f'page {page}:')
    return (bs
            .iloc[page_size * page: page_size * (page + 1)]
           [['num_unclaimed', 'name', 'set', 'qty', 'colorIdentity',
             'foil', 'mytype', 'convertedManaCost', 'price',
             'in_collections']]
           .rename(columns={'convertedManaCost': 'cmc'}))

In [ ]:
(bs
 [bs.set.isin(['KLD'])]
 [['num_unclaimed', 'name', 'set', 'qty', 'colorIdentity',
   'foil', 'mytype', 'price', 'in_collections']])

looking for secret multi-colored cards

In [ ]:
inventory = T.df_inventory()

In [ ]:
def num_unclaimed(rec):
    try:
        num_claimed = sum(_['qty']
                          for _ in
                          rec.other_collections['collections']
                          if _['url'] in T._CURRENT_DECK_URLS)
        return max(0, rec.qty - num_claimed)
    except (AttributeError, TypeError):
        return rec.qty
    except Exception as e:
        raise

inventory.loc[:, 'num_unclaimed'] = (inventory
                                     .apply(num_unclaimed, axis=1)
                                     .fillna(0))
inventory = inventory[inventory.num_unclaimed > 0].copy()

# order also depends on number of colors involved in casting; create an
# ordered category for this
def colorstr(rec):
    try:
        return ''.join(sorted(rec))
    except TypeError:
        return ''

inventory.loc[:, 'colorstr'] = inventory.colorIdentity.apply(colorstr)
inventory.colorstr = inventory.colorstr.astype('category')

def color_category_order(cat):
    """color order within the binder"""
    numcolor = len(cat)
    ismono = numcolor == 1
    iscolorless = numcolor == 0
    ismulti = numcolor > 1

    return (not ismono,
            not iscolorless,
            not ismulti,
            # just one in the opposite order
            -numcolor,
            # break ties by color
            'W' not in cat,
            'U' not in cat,
            'B' not in cat,
            'R' not in cat,
            'G' not in cat,)

orderedcats = sorted(inventory.colorstr.cat.categories,
                     key=color_category_order)
inventory.colorstr = (inventory
                      .colorstr
                      .cat
                      .reorder_categories(orderedcats, ordered=True))

# ditto for type
inventory.loc[:, 'mytype'] = (inventory
                              .type
                              .str.replace('\u2014', '-')
                              .str.replace(' - ', '|')
                              .str.extract('([^|]+)', expand=False))

inventory.replace({'mytype': {'Artifact Land': 'Land',
                              'Basic Land': 'Land',
                              'Enchantment ': '',
                              'Tribal ': '',
                              'Legendary ': '',
                              'Legendary Enchantment ': '', }, },
                  inplace=True,
                  regex=True)

inventory.mytype = inventory.mytype.astype('category')

def type_category_order(cat):
    """type order within the binder"""
    cat = cat.lower()
    return (cat != 'planeswalker',
            cat != 'creature',
            cat != 'enchantment',
            cat != 'sorcery',
            cat != 'instant',
            cat != 'artifact',
            cat != 'artifact creature',
            cat != 'land',)

orderedtypes = sorted(inventory.mytype.cat.categories,
                      key=type_category_order)
inventory.mytype = inventory.mytype.cat.reorder_categories(orderedtypes,
                                                           ordered=True)

# separate lands, even if they have color identity (otherwise they get
# sorted in with their colors)
inventory.loc[:, 'is_land'] = inventory.mytype == 'Land'

# finally, sort everything
inventory = inventory.sort_values(
    by=['is_land', 'mytype', 'convertedManaCost', 'name', 'foil'])

In [ ]:
def is_secret_multi(rec):
    # avoid split cards
    if '/' in rec['name']:
        return False
    
    # no lands
    if 'Land' in rec.type:
        return False
    
    try:
        ci = set(rec.colorIdentity)
    except TypeError:
        return False
    if len(ci) < 2:
        return False
    
    casting_costs = {c: str(rec.manaCost).count(c) for c in ci}
    return 0 in casting_costs.values()

multis = [
    'Court Hussar',
    #'Mardu Hateblade'
]
for m in multis:
    rec = inventory[inventory['name'] == m].iloc[0]
    assert is_secret_multi(rec)

In [ ]:
inventory.loc[:, 'is_secret_multi'] = (inventory
                                       .apply(is_secret_multi, axis=1))

In [ ]:
keep_keys = ['name', 'qty', 'foil', 'px', 'tla', 'type', 'tcg-foil-price',
             'colorIdentity', 'power', 'toughness', 'manaCost',
             'convertedManaCost', 'set', 'in_collections', ]
secret_multis = (inventory
                 [inventory.is_secret_multi]
                 [keep_keys])

# secret_multis.head()

In [ ]:
COLOR = 'G'

(secret_multis
 [secret_multis.manaCost.str.contains(COLOR)
  & (secret_multis.px < 0.3)])

# todo

+ figure out why "call to the grave" doesn't register
+ figure out where the "call to the grave" and "fateful showdonw" cards *should* go
+ continue where we left off (colorless bulk cards)

# dev

In [ ]:
bs.head()

In [ ]:
TROUBLESHOOT_CARDNAME = "Apostle's Blessing"

In [ ]:
bs[bs['name'].str.lower() == TROUBLESHOOT_CARDNAME.lower()]

In [ ]:
pagelength = 500
_requests = T._requests
_LOGGER = T._LOGGER
cards = T.cards


inventory = []
params = {'length': pagelength, 'start': 0, }

while True:
    resp = _requests.get(url.format(owner=owner), params=params)

    # we *should* be able to read everything returned this way
    try:
        j = resp.json()
    except _JSONDecodeError:
        print(resp.status_code)
        raise

    # j['data'] is a possibly-empty list. iterate until it's empty.
    if j['data']:
        inventory += j['data']
        params['start'] += pagelength
        _LOGGER.debug('collected {} records so far'.format(len(inventory)))
    else:
        break

# we get a bit of extra information from the mtgjson site we'd like to join
# in (specifically, cmc and color identity), so pivot that out into a more
# useful lookup dict. include a hand-maintained mapping from mtgjson set
# names to those supported in tappedout
setname_remapping = {'CMA': 'CM1'}

def parse_set_name(card):
    setname = card.get('setname')
    return setname_remapping.get(setname, setname)

mtgjson = {(card.get('name', '').lower(), parse_set_name(card)): card
           for card in cards.get_cards()}

In [ ]:
_html = T._html

for record in inventory:
    record['qty'] = record['amount']['qty']
    carddetails = _html.fromstring(record['card']).find('.//a').attrib
    record.update({k.replace('data-', ''): v
                   for (k, v) in carddetails.items()
                   if k.startswith('data-')})
    record.update(record['edit'])

    # merging with mtgjson requires fixing the setname and the card name
    setname = record['set']
    if setname == '000':
        for (_, sn) in record['all_printings']:
            if sn != setname:
                setname = sn
                break

    cardname = record['name'].lower()
    
    if cardname == TROUBLESHOOT_CARDNAME.lower():
        print(record)
        break

In [ ]:
pprint(carddetails)

In [ ]:
record.update(mtgjson.get((cardname, setname), {}))

In [ ]:
pprint(record)

In [ ]:
is_foil = record['foil'] is not None
is_foil

In [ ]:
tcg_px_key = f"tcg{'-foil' if is_foil else ''}-price"
price = float(record[tcg_px_key])

In [ ]:
price

In [ ]:
record['px'] = float(price)

In [ ]:
record['in_collections'] = {_['url'] for _ in
                                        record['other_collections'][
                                            'collections']}

In [ ]:
_pd = T._pd
get_inventory = T.get_inventory

inventory = _pd.DataFrame(get_inventory(url, owner))

In [ ]:
rec.other_collections

In [ ]:
_CURRENT_DECK_URLS = T._CURRENT_DECK_URLS
rec = inventory.loc[inventory.name.str.lower() == TROUBLESHOOT_CARDNAME.lower()].iloc[0]
num_already_claimed = sum(_['qty']
                         for _ in rec.other_collections['collections']
                         if _['url'] in _CURRENT_DECK_URLS)

In [ ]:
num_already_claimed

In [ ]:
_CURRENT_DECK_URLS = T._CURRENT_DECK_URLS

# some collections are fixed and off limits -- if a card is in one,
# we won't binder it. take the number of copies we have and subtract from
# it the number of copies in decks we are preserving
def num_unclaimed(rec):
    try:
        sacred_collections = rec.in_collections.intersection(_CURRENT_DECK_URLS)
        num_claimed = len(sacred_collections)
        return max(0, rec.qty - num_claimed)
    except AttributeError:  # rec.in_collections can be None
        return rec.qty

inventory.loc[:, 'num_unclaimed'] = inventory.apply(num_unclaimed, axis=1)
# inventory = inventory[inventory.num_unclaimed > 0]

In [ ]:
inventory[inventory['name'] == TROUBLESHOOT_CARDNAME]

In [ ]:
T._logging.basicConfig()
inventory = T.df_inventory('http://tappedout.net/api/inventory/{owner:}/board/', 'ndlambo')